We are using yasp and heppyy + alian code
First we setup the module paths etc

In [1]:
import yasp
# yasp.debug = True
import sys
import os
yasp_prefix = yasp.yasp_feature("prefix")
site_packages = os.path.join(sys.prefix, 'lib', f'python{sys.version[:3]}', 'site-packages')
add_path = os.path.join(yasp_prefix, 'lib', f'python{sys.version[:3]}', 'site-packages')
sys.path.append(add_path)

Now we load the environment modules and setup paths for python

In [2]:
yasp.module_load_cppyy('bundle/hepbase')
yasp.module_load_cppyy('heppyy/current')
yasp.module_load_cppyy('alian/current')

import heppyy
fj = heppyy.load_cppyy('fastjet')
std = heppyy.load_cppyy('std')
alian_cpp = heppyy.load_cppyy('alian')

/Users/ploskon/devel/yasp/software/bundle/hepbase/root/default/lib/cppyy_backend/loader.py:90: UserWarning: No precompiled header available (/Users/ploskon/devel/yasp/software/bundle/hepbase/root/default/lib/cppyy_backend/etc not writable); this may impact performance.
  warnings.warn('No precompiled header available (%s); this may impact performance.' % msg)
[yasp-i] Running in Jupyter: True


In [3]:
import alian

In [4]:
fj.ClusterSequence().print_banner()

jet_algorithm = fj.antikt_algorithm
jet_R = 0.4
jet_eta_max = 0.5
bg_y_max = 0.9
bg_grid_spacing = 0.1

jet_def = fj.JetDefinition(jet_algorithm, jet_R)
area_def = fj.AreaDefinition(fj.active_area, fj.GhostedAreaSpec(jet_eta_max + jet_R, 1, 0.01))
jet_selector = fj.SelectorAbsEtaMax(jet_eta_max) * fj.SelectorPtMin(20.0)
bg_estimator = fj.GridMedianBackgroundEstimator(bg_y_max, bg_grid_spacing)

#--------------------------------------------------------------------------
#                         FastJet release 3.4.4-devel
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code
# and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------


In [5]:
pt_hat_min = 100
pythia_cmnd = os.path.join(alian.alian_settings.src_path, 'config/pythia-pp-hardQCD-5TeV-Monash.cmnd')
pythia_settings = ['PhaseSpace:pThatMin = {}'.format(pt_hat_min)]

# Pythia8 = heppyy.load_cppyy('pythia8.Pythia8')

## from alian.io.pythia_io import PythiaInput
## pythia_input = PythiaInput(pythia_cmnd_file=pythia_cmnd, user_settings=pythia_settings, auto_next=True)

In [6]:
pt_hat_min = 100

pythia_cmnd = os.path.join(alian.alian_settings.src_path, 'config/pythia-pp-hardQCD-5TeV-Monash.cmnd')
pythia_settings = ['PhaseSpace:pThatMin = {}'.format(pt_hat_min)]

from heppyy.pythia_util.configuration import create_and_init_pythia
pythia = create_and_init_pythia(pythia_settings, pythia_cmnd)


 *------------------------------------------------------------------------------------* 
 |                                                                                    | 
 |  *------------------------------------------------------------------------------*  | 
 |  |                                                                              |  | 
 |  |                                                                              |  | 
 |  |   PPP   Y   Y  TTTTT  H   H  III    A      Welcome to the Lund Monte Carlo!  |  | 
 |  |   P  P   Y Y     T    H   H   I    A A     This is PYTHIA version 8.310      |  | 
 |  |   PPP     Y      T    HHHHH   I   AAAAA    Last date of change: 25 Jul 2023  |  | 
 |  |   P       Y      T    H   H   I   A   A                                      |  | 
 |  |   P       Y      T    H   H  III  A   A    Now is 15 May 2025 at 21:05:56    |  | 
 |  |                                                                              |  | 
 |  |   Program docu

In [7]:
from alian.io.pythia_io import psj_from_particle_with_index
pythia_offset_index = 0
i = 0
while pythia.next():
    print(f'Event: {i} nparts: {pythia.event.size()}')
    i += 1
    psjv = std.vector[fj.PseudoJet]([psj_from_particle_with_index(p, i + pythia_offset_index) 
                                    for i, p in enumerate(pythia.event) if p.isFinal() and p.isVisible() and p.isCharged()])
    ca = fj.ClusterSequenceArea(psjv, jet_def, area_def)
    jets = fj.sorted_by_pt(jet_selector(ca.inclusive_jets()))
    leadpt = -1
    if jets.size() > 0:
        leadpt = jets[0].perp()
    else:
        continue
    print(f'{i} : n particles: {psjv.size()} -> njets: {jets.size()} | highest pt: {leadpt}')
    i += 1
    if i > 100:
        break    

[i-PythiaInput] Running in Jupyter Notebook, using tqdm.notebook
Event: 0 nparts: 742
1 : n particles: 114 -> njets: 1 | highest pt: 24.732513077604455
Event: 2 nparts: 520
Event: 3 nparts: 911
Event: 4 nparts: 1334
Event: 5 nparts: 625
Event: 6 nparts: 983
7 : n particles: 168 -> njets: 1 | highest pt: 76.57580131865012
Event: 8 nparts: 451
9 : n particles: 64 -> njets: 1 | highest pt: 91.14478470130761
Event: 10 nparts: 1050
Event: 11 nparts: 1070
12 : n particles: 170 -> njets: 1 | highest pt: 85.27095792375732
Event: 13 nparts: 1636
Event: 14 nparts: 882
15 : n particles: 122 -> njets: 2 | highest pt: 117.2013035877239
Event: 16 nparts: 735
Event: 17 nparts: 1769
Event: 18 nparts: 999
19 : n particles: 160 -> njets: 1 | highest pt: 91.69635852857098
Event: 20 nparts: 1676
Event: 21 nparts: 1467
Event: 22 nparts: 1942
Event: 23 nparts: 1216
24 : n particles: 176 -> njets: 1 | highest pt: 34.21520075035029
Event: 25 nparts: 295
Event: 26 nparts: 1031
27 : n particles: 166 -> njets: 1